In [ ]:
from nbdev import *

In [ ]:
#hide
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

# DCAE Hyperparameter sweep

> Run a hyperparamter sweep for the Deep Convolutional AutoEncoder defined in the notebook `01_Timecluster_replication.ipynb`

This sweep has been run using Weights & Biases, following [this tutorial](https://www.wandb.com/articles/running-hyperparameter-sweeps-to-pick-the-best-model-using-w-b).

In [ ]:
#hide
import wandb
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
from timecluster_extension.all import *
from functools import partial
from fastcore.all import *

### Sweep configuration 

In [ ]:
# Select the hyperparameters you want to tune. This is specified like the following:
sweep_config = {
    'name': 'DCAE-JNK-v6-bayes',
   'method': 'bayes',
    "metric": {
        "name" : "val_loss",
        "goal" : "minimize"
    },
   'parameters': {
       'w': {
           'values': list(range(12, 144, 12))
       },
        'stride': {
           'values': [1, 5, 10, 20]
       },
       'lr': {
           'distribution': 'uniform',
           'min': 0.00001,
           'max': 0.01
       },
       'epochs': {
           'distribution': 'int_uniform',
           'min': 1,
           'max': 50
       },
       'batch_size': {
           'distribution': 'int_uniform',
           'min': 32,
           'max': 256
       },
       'n_filters': {
           'values': [[64,32,12],[32,16,12]]
       },
       'filter_sizes': {
           'values': [[10, 5, 5], [20, 10, 10], [30, 15, 15]]
       },
       'output_filter_size': {
           'distribution': 'int_uniform',
           'min': 5,
           'max': 30
       },
       'pool_sizes': {
           'value': [2, 2, 3]
       }
   },
    'early_terminate': {
        'type': 'hyperband',
        's': 2,
        'eta': 3,
        'max_iter': 27
    }
}

In [ ]:
# Select the hyperparameters you want to tune. This is specified like the following:
sweep_config_lr = {
    'name': 'DCAE-JNK-v6-bayes',
   'method': 'bayes',
    "metric": {
        "name" : "val_loss",
        "goal" : "minimize"
    },
   'parameters': {
       'lr': {
           'distribution': 'uniform',
           'min': 0.00001,
           'max': 0.01
       }
   },
    'early_terminate': {
        'type': 'hyperband',
        's': 2,
        'eta': 3,
        'max_iter': 27
    }
}

### Define the agent function (train)

Once you have initialized the sweep you need an agent. An agent is a model training script you can use to pair the sweep configurations. Let’s define a simple training script:

In [ ]:
def train(dataset):
    # Specify the hyperparameter to be tuned along with an initial value
    config_defaults = {
        'lr': 0.002438113880973422,
        'n_filters': [32, 16, 12],
        'filter_sizes': [10, 5, 5],
        'output_filter_size': 10,
        'pool_sizes': [2, 2, 3],
        'batch_size': 75,
        'epochs': 34,
        'stride': 1,
        'w': 60
    }
    # Initialize wandb with a sample project name
    run = wandb.init(project="timecluster-extension", 
                     job_type='train_DCAE', 
                     allow_val_change=True, 
                     config=config_defaults)
    
    # Load and preprocess the data
    ds_artifact = run.use_artifact(dataset, type='dataset')
    df = ds_artifact.to_df() # comes from TSArtifact
    input_data = df_slicer(df, w=wandb.config.w, s=wandb.config.stride)

    # Specify the other hyperparameters to the configuration
    wandb.config.update({
                            'ds_artifact_type': ds_artifact.type,
                            'ds_artifact_name': ds_artifact.name,
                            'ds_artifact_digest': ds_artifact.digest,
                            'val_pct': 0.2
                        }, allow_val_change=True)

    # Define the model
    model = createDCAE(w=wandb.config.w, 
                       d=input_data.shape[2], 
                       delta=wandb.config.w, 
                       n_filters=wandb.config.n_filters, 
                       filter_sizes=wandb.config.filter_sizes, 
                       pool_sizes=wandb.config.pool_sizes, 
                       output_filter_size=wandb.config.output_filter_size)

    opt = Adam(learning_rate=wandb.config.lr)

    # Compile the model
    model.compile(optimizer=opt,
                 loss='mean_squared_error',
                 metrics=['mean_squared_error'])

    # Train the model
    model.fit(x=input_data, y=input_data, batch_size=wandb.config.batch_size, 
      validation_split=wandb.config.val_pct, epochs=wandb.config.epochs, verbose=0, 
      callbacks=[WandbCallback()])

### Create the sweep and call the training function from a wandb agent 

In [ ]:
sweep_id = wandb.sweep(sweep_config_lr)

Create sweep with ID: rjmxj4uh
Sweep URL: https://app.wandb.ai/vrodriguezf/timecluster-extension/sweeps/rjmxj4uh


In [ ]:
wandb.agent(sweep_id, function=partial(train, dataset='JNK:v6'))

wandb: Agent Starting Run: pdap1i5u with config:
	lr: 0.005344555288972418
wandb: Agent Started Run: pdap1i5u


wandb: Wandb version 0.9.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Downloading large artifact JNK:v6, 65.00MB. 1 files... Done. 0.3s
